# Network Connectivity

This notebook demonstrates network connectivity analysis using NetworkX.

## Learning Objectives
- Understand walks, paths, and connected components
- Analyze network connectivity measures
- Study giant component analysis
- Explore assortativity and degree correlations

In [ ]:
# Import required libraries
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# For better display in notebooks
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Walks and Paths

Let's start by understanding the difference between walks and paths in networks.

In [ ]:
# Create a simple graph to demonstrate walks and paths
G = nx.Graph()
edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 1), (2, 4)]
G.add_edges_from(edges)

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Find all simple paths between nodes 1 and 5
paths = list(nx.all_simple_paths(G, 1, 5))
print(f"\nAll simple paths from node 1 to node 5:")
for i, path in enumerate(paths, 1):
    print(f"Path {i}: {path}")

# Find shortest path
shortest_path = nx.shortest_path(G, 1, 5)
print(f"\nShortest path from node 1 to node 5: {shortest_path}")
print(f"Shortest path length: {len(shortest_path) - 1}")

In [ ]:
# Visualize the graph with the shortest path highlighted
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Draw the graph
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=12, font_weight='bold')

# Highlight the shortest path
path_edges = list(zip(shortest_path[:-1], shortest_path[1:]))
nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', 
                        width=3, alpha=0.7)

plt.title('Graph with Shortest Path Highlighted', fontsize=14, fontweight='bold')
plt.show()

## 2. Connected Components

Let's analyze connected components in networks.

In [ ]:
# Create a graph with multiple components
G_multi = nx.Graph()
edges_multi = [(1, 2), (2, 3), (3, 1),  # Component 1
               (4, 5), (5, 6), (6, 4),  # Component 2
               (7, 8), (8, 9), (9, 7)]  # Component 3
G_multi.add_edges_from(edges_multi)

# Find connected components
components = list(nx.connected_components(G_multi))
print(f"Number of connected components: {len(components)}")
print(f"\nComponents:")
for i, component in enumerate(components, 1):
    print(f"Component {i}: {sorted(component)}")
    print(f"  Size: {len(component)} nodes")

# Check if graph is connected
print(f"\nIs the graph connected? {nx.is_connected(G_multi)}")

# Find the largest connected component (giant component)
giant_component = max(components, key=len)
print(f"\nGiant component: {sorted(giant_component)}")
print(f"Giant component size: {len(giant_component)} nodes")

In [ ]:
# Visualize the multi-component graph
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G_multi, seed=42)

# Color nodes by component
colors = []
for node in G_multi.nodes():
    for i, component in enumerate(components):
        if node in component:
            colors.append(i)
            break

nx.draw(G_multi, pos, with_labels=True, node_color=colors, 
        cmap=plt.cm.Set3, node_size=500, font_size=12, font_weight='bold')

plt.title('Multi-Component Graph', fontsize=14, fontweight='bold')
plt.show()

## 3. Node and Edge Connectivity

Let's analyze the connectivity of networks.

In [ ]:
# Create a more complex graph for connectivity analysis
G_connect = nx.Graph()
edges_connect = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 1),  # Main cycle
                 (2, 6), (6, 7), (7, 8), (8, 3),  # Bridge
                 (9, 10), (10, 11), (11, 9)]  # Separate component
G_connect.add_edges_from(edges_connect)

print(f"Graph: {G_connect.number_of_nodes()} nodes, {G_connect.number_of_edges()} edges")

# Node connectivity
node_connectivity = nx.node_connectivity(G_connect)
print(f"\nNode connectivity: {node_connectivity}")

# Edge connectivity
edge_connectivity = nx.edge_connectivity(G_connect)
print(f"Edge connectivity: {edge_connectivity}")

# Find articulation points (cut vertices)
articulation_points = list(nx.articulation_points(G_connect))
print(f"\nArticulation points: {articulation_points}")

# Find bridges
bridges = list(nx.bridges(G_connect))
print(f"Bridges: {bridges}")

## 4. Giant Component Analysis

Let's study the emergence of giant components in random networks.

In [ ]:
# Simulate giant component emergence
def analyze_giant_component(n_nodes, p_values):
    """Analyze giant component size for different edge probabilities"""
    results = []
    
    for p in p_values:
        # Create random graph
        G_rand = nx.erdos_renyi_graph(n_nodes, p)
        
        # Find largest component
        components = list(nx.connected_components(G_rand))
        if components:
            largest_component_size = max(len(comp) for comp in components)
            giant_component_fraction = largest_component_size / n_nodes
        else:
            giant_component_fraction = 0
        
        results.append({
            'p': p,
            'giant_component_fraction': giant_component_fraction,
            'n_components': len(components)
        })
    
    return results

# Test different edge probabilities
n_nodes = 100
p_values = np.linspace(0.001, 0.1, 20)
results = analyze_giant_component(n_nodes, p_values)

# Create DataFrame
df_results = pd.DataFrame(results)
print("Giant Component Analysis Results:")
print(df_results.head(10))

In [ ]:
# Plot giant component emergence
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(df_results['p'], df_results['giant_component_fraction'], 'bo-')
plt.xlabel('Edge Probability (p)')
plt.ylabel('Giant Component Fraction')
plt.title('Giant Component Emergence')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_results['p'], df_results['n_components'], 'ro-')
plt.xlabel('Edge Probability (p)')
plt.ylabel('Number of Components')
plt.title('Number of Connected Components')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Assortativity Analysis

Let's analyze degree assortativity in networks.

In [ ]:
# Load a real network for assortativity analysis
G_real = nx.les_miserables_graph()
print(f"Les Miserables network: {G_real.number_of_nodes()} nodes, {G_real.number_of_edges()} edges")

# Calculate degree assortativity
assortativity = nx.degree_assortativity_coefficient(G_real)
print(f"\nDegree assortativity: {assortativity:.4f}")

# Interpret the result
if assortativity > 0:
    print("The network is assortative (high-degree nodes connect to high-degree nodes)")
elif assortativity < 0:
    print("The network is disassortative (high-degree nodes connect to low-degree nodes)")
else:
    print("The network shows no degree correlation")

# Calculate degree distribution
degrees = [d for n, d in G_real.degree()]
print(f"\nDegree statistics:")
print(f"Mean degree: {np.mean(degrees):.2f}")
print(f"Standard deviation: {np.std(degrees):.2f}")
print(f"Minimum degree: {min(degrees)}")
print(f"Maximum degree: {max(degrees)}")

In [ ]:
# Visualize degree distribution and assortativity
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Degree distribution
ax1.hist(degrees, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_xlabel('Degree')
ax1.set_ylabel('Frequency')
ax1.set_title('Degree Distribution')
ax1.grid(True, alpha=0.3)

# Degree-degree correlation
degrees_array = np.array(degrees)
ax2.scatter(degrees_array[:-1], degrees_array[1:], alpha=0.6, color='red')
ax2.set_xlabel('Degree of Node i')
ax2.set_ylabel('Degree of Node i+1')
ax2.set_title('Degree-Degree Correlation')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Summary and Key Insights

### Key Takeaways:

1. **Walks vs Paths**: Walks can repeat nodes, paths cannot
2. **Connected Components**: Groups of nodes that can reach each other
3. **Giant Component**: The largest connected component in a network
4. **Connectivity Measures**: Node and edge connectivity indicate network robustness
5. **Assortativity**: Measures degree correlation between connected nodes

### Applications:
- **Social Networks**: Understanding community structure
- **Infrastructure**: Analyzing network resilience
- **Biological Networks**: Studying functional modules
- **Information Networks**: Understanding information flow